In [1]:
# LLM Simples para Tradução Português-Guarani
# Versão Simplificada para Google Colab

# ============================================================================
# 1. INSTALAÇÃO
# ============================================================================

!pip install transformers torch datasets evaluate

import torch
import pandas as pd
from transformers import (
    GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from datasets import Dataset
import evaluate

print("✅ Setup completo!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# ============================================================================
# 2. DADOS
# ============================================================================
import random
import pandas as pd
from datasets import Dataset, concatenate_datasets
from google.colab import drive
drive.mount('/content/drive')
# Caminho para o arquivo JSON
data_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json"
df = pd.read_json(data_path)

# Lista para armazenar dados aumentados no novo formato

# ✅ Instruções para aumento de dados
# Instruções variadas
instructions_pt = [
    "Traduza para o português:", "Tradução em português:", "O que significa em português:",
    "Traduza do Guarani para o Português:", "Como se diz em português:", "Converta para o português:",
    "Passe para o português:", "A tradução portuguesa é:", "Explique em português:",
    "Diga isso em português:", "Interprete isso em português:", "Tradução correta em português:",
    "Versão em português:", "Tradução literal para o português:", "Tradução aproximada em português:",
    "Tradução com sentido cultural em português:"
]
instructions_gua = [
    "Traduza para o guarani:", "Versão em Guarani:", "O que significa em Guarani:",
    "Tradução em Guarani:", "Como se diz em Guarani:", "Com respeito à cultura Guarani",
    "Em guarani, significa:", "Explique em Guarani:", "Tradução para o Guarani tradicional:",
    "Tradução para o Guarani moderno:", "Tradução Jopara:", "Versão Guarani com contexto cultural:",
    "Diga isso em Guarani:", "Converta para Guarani:", "Traduzindo para Guarani com respeito:",
    "Tradução respeitosa em Guarani:"
]

# Aumento de dados
augmented_data = []
for _, row in df.iterrows():
    for _ in range(5):
        inst = random.choice(instructions_pt)
        augmented_data.append({
            "instruction": inst,
            "input": row["input"],
            "output": row["output"]
        })
        inst_inv = random.choice(instructions_gua)
        augmented_data.append({
            "instruction": inst_inv,
            "input": row["output"],
            "output": row["input"]
        })

print(f"✅ Dataset aumentado: {len(augmented_data)} exemplos")

df_augmented = pd.DataFrame(augmented_data)
df_augmented = df_augmented.dropna()

dataset_augmented = Dataset.from_pandas(df_augmented)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1373 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
# ============================================================================
# 3. FORMATAÇÃO
# ============================================================================
def format_example(item):
    return f"Pergunta: {item['instruction']}\nEntrada: {item['input']}\nResposta: {item['output']}"

texts = [format_example(item) for item in augmented_data]

# Dividir treino/teste (80/20)
split = int(0.8 * len(texts))
train_texts = texts[:split]
test_texts = texts[split:]

print(f"🚂 Treino: {len(train_texts)} | Teste: {len(test_texts)}")

In [ ]:
# ============================================================================
# 4. TOKENIZER E MODELO
# ============================================================================

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Modelo pequeno
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=128,
    n_embd=256,
    n_layer=4,
    n_head=4
)

model = GPT2LMHeadModel(config)
print(f"🤖 Modelo: {model.num_parameters():,} parâmetros")


In [ ]:
# ============================================================================
# 5. DATASET
# ============================================================================

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_dataset = Dataset.from_dict({'text': train_texts})
test_dataset = Dataset.from_dict({'text': test_texts})

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# ============================================================================
# 6. TREINAMENTO
# ============================================================================

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./modelo-guarani',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

print("🚀 Iniciando treinamento...")
trainer.train()
print("✅ Treinamento concluído!")

# ============================================================================
# 7. TESTE E AVALIAÇÃO
# ============================================================================

# Função para gerar resposta com prompt no estilo Instruct
def gerar_resposta(instruction, entrada=""):
    prompt = f"### Instrução:\n{instruction}\n"
    if entrada.strip():
        prompt += f"\n### Entrada:\n{entrada}\n"
    prompt += "\n### Resposta:\n"

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=inputs['input_ids'].shape[1] + 20,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta.replace(prompt, "").strip()
# ============================================================================
# 7.1 AVALIAÇÃO COM DADOS AUMENTADOS
# ============================================================================

# Usa o dataset aumentado usado no treinamento/validação
avaliacao_data = augmented_data  # ou use uma parte do train_data/val_data se preferir

print("\n🧪 TESTES:")
print("="*50)

acertos = 0
total = len(avaliacao_data)

for item in avaliacao_data[:30]:  # limitar a 30 testes para agilizar
    pergunta = item['instruction']
    entrada = item.get('input', "")
    esperado = item['output']
    gerado = gerar_resposta(pergunta, entrada)

    correto = esperado.lower().strip() in gerado.lower()
    if correto:
        acertos += 1

    print(f"\n❓ {pergunta}")
    print(f"📥 Entrada: {entrada}")
    print(f"✅ Esperado: {esperado}")
    print(f"🤖 Gerado: {gerado}")
    print(f"{'✅ CORRETO' if correto else '❌ ERRADO'}")
# ============================================================================
# 8. MÉTRICAS
# ============================================================================

acuracia = acertos / total
print(f"\n📊 RESULTADOS:")
print(f"Acurácia: {acuracia:.2%} ({acertos}/{total})")
# ============================================================================
# 9. TESTE INTERATIVO
# ============================================================================
print("\n🎯 TESTE INTERATIVO (respostas em espanhol sobre o livro Ñande Ypykuéra)")
print("Digite uma pergunta (ou 'sair'):")

while True:
    pergunta = input("\n👤 Pergunta sobre o livro: ")
    if pergunta.lower() in ['sair', 'exit', 'quit']:
        break

    resposta = gerar_resposta("Responda sobre o livro 'Ñande Ypykuéra'", pergunta)
    print(f"🤖 Respuesta: {resposta}")
